In [5]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import os
import sys
import csv
import time
import ccxt  # noqa: E402

In [6]:
def retry_fetch_ohlcv(exchange, max_retries, symbol, timeframe, since, limit):
    """
    Retry fetching OHLCV data
    """
    num_retries = 0
    try:
        num_retries += 1
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since, limit)
        return ohlcv
    except Exception as e:
        if num_retries > max_retries:
            print(f"Failed to fetch data: {e}")
            raise
        print(f"Retrying to fetch data ({num_retries}/{max_retries})...")
        time.sleep(exchange.rateLimit / 1000)  # Sleep before retrying
        return retry_fetch_ohlcv(exchange, max_retries - 1, symbol, timeframe, since, limit)

In [7]:
def scrape_ohlcv(exchange, max_retries, symbol, timeframe, since, limit, end_time=None):
    """
    Scrape OHLCV data until current time or end time
    """
    timeframe_duration_in_seconds = exchange.parse_timeframe(timeframe)
    timeframe_duration_in_ms = timeframe_duration_in_seconds * 1000
    timedelta = limit * timeframe_duration_in_ms
    
    # If no end time specified, use current time
    if end_time is None:
        end_time = exchange.milliseconds()
    
    all_ohlcv = []
    fetch_since = since
    
    while fetch_since < end_time:
        print(f"Fetching {symbol} {timeframe} data from {exchange.iso8601(fetch_since)}")
        ohlcv = retry_fetch_ohlcv(exchange, max_retries, symbol, timeframe, fetch_since, limit)
        
        # If no data received, move time window and continue
        if len(ohlcv) == 0:
            fetch_since = fetch_since + timedelta
            continue
            
        # Update next fetch start time
        fetch_since = ohlcv[-1][0] + 1
        
        all_ohlcv.extend(ohlcv)
        print(f"Retrieved {len(all_ohlcv)} candlesticks from {exchange.iso8601(all_ohlcv[0][0])} to {exchange.iso8601(all_ohlcv[-1][0])}")
        
        # Avoid too frequent requests
        time.sleep(exchange.rateLimit / 1000)
    
    # Filter results to ensure within specified time range
    return exchange.filter_by_since_limit(all_ohlcv, since, end_time, key=0)

In [8]:
def download_crypto_data(exchange_id, symbols, timeframes, start_dates, end_dates):
    # Initialize exchange with increased timeout and proxy settings
    exchange = getattr(ccxt, exchange_id)({
        'enableRateLimit': True,
        'timeout': 30000,
        'options': {
            'timeout': 30000,
        },
        'proxies': {
            'http': 'http://127.0.0.1:7890',
            'https': 'http://127.0.0.1:7890',
        }
    })

    # Add retry mechanism for market loading
    max_load_retries = 3
    retry_delay = 5

    for attempt in range(max_load_retries):
        try:
            exchange.load_markets()
            break
        except Exception as e:
            if attempt == max_load_retries - 1:
                print(f"Failed to load markets after {max_load_retries} attempts: {e}")
                raise
            print(f"Failed to load markets, retrying in {retry_delay} seconds...")
            time.sleep(retry_delay)

    # Set maximum number of candlesticks per request
    limit = 1000  # Most exchanges limit to 1000
    max_retries = 3
    all_data = {}

    for symbol in symbols:
        symbol_data = {}
        for i, timeframe in enumerate(timeframes):
            # Convert date strings to millisecond timestamps
            since = exchange.parse8601(start_dates[i])
            end_time = None if end_dates[i] is None else exchange.parse8601(end_dates[i])

            print(f"\nStarting to download {exchange_id} {symbol} {timeframe} data...")
            print(f"Time range: {start_dates[i]} to {end_dates[i] if end_dates[i] else 'now'}")

            try:
                # Fetch candlestick data
                ohlcv = scrape_ohlcv(exchange, max_retries, symbol, timeframe, since, limit, end_time)
                symbol_data[timeframe] = ohlcv
                print(f"Successfully downloaded {symbol} {timeframe} data")
            except Exception as e:
                print(f"Failed to download {symbol} {timeframe} data: {e}")
                symbol_data[timeframe] = None

            # Add delay between different trading pairs and timeframes to avoid rate limiting
            time.sleep(1)
        all_data[symbol] = symbol_data

    return all_data

In [9]:

    # Exchange ID
    exchange_id = 'binance'

    # Trading pairs
    symbols = ['BTC/USDT']

    # Time periods
    timeframes = ['1h']

    # Start dates (corresponding to each timeframe)
    start_dates = ['2024-01-01T00:00:00Z', '2024-01-01T00:00:00Z']

    # End dates (corresponding to each timeframe)
    # Daily data until now
    end_dates = [None, '2025-03-31T23:59:59Z']



In [10]:
    # 调用download_BTC_data_binance中的fetch_crypto_data函数获取原始数据
    ohlcv_data = download_crypto_data(exchange_id, symbols, timeframes, start_dates, end_dates)
    # print(ohlcv_data)
    # 将OHLCV数据转换为DataFrame（格式需与process_raw_data的输入匹配）
   


Starting to download binance BTC/USDT 1h data...
Time range: 2024-01-01T00:00:00Z to now
Fetching BTC/USDT 1h data from 2024-01-01T00:00:00.000Z
Retrieved 1000 candlesticks from 2024-01-01T00:00:00.000Z to 2024-02-11T15:00:00.000Z
Fetching BTC/USDT 1h data from 2024-02-11T15:00:00.001Z
Retrieved 2000 candlesticks from 2024-01-01T00:00:00.000Z to 2024-03-24T07:00:00.000Z
Fetching BTC/USDT 1h data from 2024-03-24T07:00:00.001Z
Retrieved 3000 candlesticks from 2024-01-01T00:00:00.000Z to 2024-05-04T23:00:00.000Z
Fetching BTC/USDT 1h data from 2024-05-04T23:00:00.001Z
Retrieved 4000 candlesticks from 2024-01-01T00:00:00.000Z to 2024-06-15T15:00:00.000Z
Fetching BTC/USDT 1h data from 2024-06-15T15:00:00.001Z
Retrieved 5000 candlesticks from 2024-01-01T00:00:00.000Z to 2024-07-27T07:00:00.000Z
Fetching BTC/USDT 1h data from 2024-07-27T07:00:00.001Z
Retrieved 6000 candlesticks from 2024-01-01T00:00:00.000Z to 2024-09-06T23:00:00.000Z
Fetching BTC/USDT 1h data from 2024-09-06T23:00:00.001Z
Re

In [11]:
ohlcv_data

{'BTC/USDT': {'1h': [[1704067200000,
    42283.58,
    42554.57,
    42261.02,
    42475.23,
    1271.68108],
   [1704070800000, 42475.23, 42775.0, 42431.65, 42613.56, 1196.37856],
   [1704074400000, 42613.57, 42638.41, 42500.0, 42581.1, 685.2198],
   [1704078000000, 42581.09, 42586.64, 42230.08, 42330.49, 794.80391],
   [1704081600000, 42330.5, 42399.99, 42209.46, 42399.99, 715.4176],
   [1704085200000, 42399.98, 42406.0, 42180.77, 42234.01, 736.53152],
   [1704088800000, 42234.01, 42424.82, 42208.68, 42396.69, 601.3725],
   [1704092400000, 42396.69, 42500.0, 42396.68, 42492.46, 653.67718],
   [1704096000000, 42492.46, 42556.5, 42452.58, 42549.99, 427.19365],
   [1704099600000, 42550.0, 42699.32, 42537.18, 42649.69, 879.62491],
   [1704103200000, 42649.69, 42749.36, 42628.0, 42691.1, 856.81851],
   [1704106800000, 42691.1, 42762.39, 42605.21, 42690.2, 785.16567],
   [1704110400000, 42690.21, 42780.0, 42611.45, 42648.38, 785.52851],
   [1704114000000, 42648.37, 42748.0, 42612.76, 42715

In [12]:
import pandas as pd

In [60]:
df_1h = pd.DataFrame(ohlcv_data['BTC/USDT']['1h'], columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])

In [70]:
df_1h

,timestamp,open,high,low,close,volume
datetime,,,,,,
2024-01-01 00:00:00,1704067200000,42283.58,42554.57,42261.02,42475.23,1271.68108
2024-01-01 01:00:00,1704070800000,42475.23,42775.00,42431.65,42613.56,1196.37856
2024-01-01 02:00:00,1704074400000,42613.57,42638.41,42500.00,42581.10,685.21980
2024-01-01 03:00:00,1704078000000,42581.09,42586.64,42230.08,42330.49,794.80391
2024-01-01 04:00:00,1704081600000,42330.50,42399.99,42209.46,42399.99,715.41760
...,...,...,...,...,...,...
2025-04-20 02:00:00,1745114400000,85320.16,85320.76,85202.92,85226.44,158.61851
2025-04-20 03:00:00,1745118000000,85226.44,85266.38,85111.00,85168.81,146.25197
2025-04-20 04:00:00,1745121600000,85168.82,85248.00,85104.85,85185.13,119.07773


# split data

In [71]:
data=df_1h
data['datetime'] = pd.to_datetime(data['timestamp'], unit='ms')
data.set_index('datetime', inplace=True)

# 定义时间范围
start_date = pd.to_datetime("2024-01-01")
end_date = pd.to_datetime("2025-04-01")
data = data[(data.index > start_date) & (data.index < end_date)]
# 计算总样本数和测试集大小（最后 20%）
total_samples = len(data)
test_size = int(total_samples * 0.2)
test_data = data.iloc[-test_size:]  

In [72]:
raw_data=test_data

In [76]:
raw_data

,timestamp,open,high,low,close,volume,price_range,price_change_pct,sma_4,volume_change,...,hour_cos,dayofweek_sin,dayofweek_cos,is_trading_day,macd,macd_signal,macd_diff,momentum,volatility_vol,return_vol_corr
datetime,,,,,,,,,,,,,,,,,,,,,
2024-12-30 20:00:00,1735588800000,94417.04,95024.50,94296.75,94338.80,1146.04057,727.75,NaN,NaN,NaN,...,0.500000,0.000000,1.00000,1,NaN,NaN,NaN,NaN,8.340310e+05,NaN
2024-12-30 21:00:00,1735592400000,94338.80,94376.30,91901.57,92076.00,2274.79849,2474.73,-2.398589,NaN,0.984920,...,0.707107,0.000000,1.00000,1,NaN,NaN,NaN,NaN,5.629512e+06,NaN
2024-12-30 22:00:00,1735596000000,92076.00,93027.69,91948.00,92784.75,1170.61959,1079.69,0.769745,NaN,-0.485396,...,0.866025,0.000000,1.00000,1,NaN,NaN,NaN,NaN,1.263906e+06,NaN
2024-12-30 23:00:00,1735599600000,92784.76,92893.40,92389.28,92792.05,578.50584,504.12,0.007868,92997.9000,-0.505812,...,0.965926,0.000000,1.00000,1,NaN,NaN,NaN,NaN,2.916364e+05,NaN
2024-12-31 00:00:00,1735603200000,92792.05,92907.03,92375.15,92450.00,584.04771,531.88,-0.368620,92525.7000,0.009580,...,1.000000,0.781831,0.62349,1,NaN,NaN,NaN,NaN,3.106433e+05,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-31 19:00:00,1743447600000,83291.42,83443.38,82400.00,82462.27,798.20925,1043.38,-0.995481,82930.7300,-0.015445,...,0.258819,0.000000,1.00000,1,268.343269,-28.538625,296.881895,0.003594,8.328356e+05,0.136567
2025-03-31 20:00:00,1743451200000,82462.26,82846.20,82350.84,82431.32,701.19812,495.36,-0.037532,82716.3375,-0.121536,...,0.500000,0.000000,1.00000,1,324.165000,52.485862,271.679138,0.004613,3.473455e+05,0.224515
2025-03-31 21:00:00,1743454800000,82431.32,82752.18,82422.00,82563.99,342.52865,330.18,0.160946,82687.2500,-0.511509,...,0.707107,0.000000,1.00000,1,415.948654,131.816204,284.132450,0.007695,1.130961e+05,0.018315


# ML

In [89]:
import pandas as pd
import numpy as np
import joblib
import pickle
from datetime import datetime, timedelta
import os

MODEL_PATH = "D:/Study_Master/TERM2/FTEC5530/ftec5530-main 1/best_model.pkl"
FEATURES_PATH = "D:/Study_Master/TERM2/FTEC5530/ftec5530-main 1/selected_features.pkl"

best_model = joblib.load(MODEL_PATH)
EXPECTED_FEATURES=['datetime','open', 'high', 'low', 'close', 'volume']

C:\Users\zyc\anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator StandardScaler from version 1.3.0 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\zyc\anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator PCA from version 1.3.0 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\zyc\anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator Pipeline from version 1.3.0 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For 

In [90]:
EXPECTED_FEATURES

['datetime', 'open', 'high', 'low', 'close', 'volume']

In [91]:
def process_raw_data(raw_df1: pd.DataFrame) -> pd.DataFrame:
    """
    将原始数据转换为模型所需的特征矩阵（与训练时完全一致）
    :param raw_df: 包含API原始数据的DataFrame
    :return: 处理后的特征矩阵（用于模型预测）
    """
    raw_df=raw_df1
    # ====================== A. 基础数据处理 ======================
    # 将时间戳转换为datetime并设置索引
    raw_df['datetime'] = pd.to_datetime(raw_df['timestamp'], unit='ms')
    raw_df.set_index('datetime', inplace=True)
    
    # ====================== B. 价格相关特征 ======================
    # 价格波动范围
    raw_df['price_range'] = raw_df['high'] - raw_df['low']
    # 涨跌幅（百分比变化）
    raw_df['price_change_pct'] = raw_df['close'].pct_change() * 100
    # 简单移动平均（4小时窗口）
    raw_df['sma_4'] = raw_df['close'].rolling(window=4).mean()
    # 交易量变化
    raw_df['volume_change'] = raw_df['volume'].pct_change()
    
    # ====================== C. 时间特征 ======================
    raw_df['hour'] = raw_df.index.hour
    raw_df['day_of_week'] = raw_df.index.dayofweek  # Monday=0, Sunday=6
    raw_df['month'] = raw_df.index.month
    raw_df['quarter'] = raw_df.index.quarter
    
    # ====================== D. 交易量相关特征 ======================
    raw_df['volume_sma_4'] = raw_df['volume'].rolling(window=4).mean()
    raw_df['volume_std_4'] = raw_df['volume'].rolling(window=4).std()
    
    # ====================== E. 高级时间特征（正弦/余弦编码） ======================
    raw_df['hour_sin'] = np.sin(2 * np.pi * raw_df['hour'] / 24)
    raw_df['hour_cos'] = np.cos(2 * np.pi * raw_df['hour'] / 24)
    raw_df['dayofweek_sin'] = np.sin(2 * np.pi * raw_df['day_of_week'] / 7)
    raw_df['dayofweek_cos'] = np.cos(2 * np.pi * raw_df['day_of_week'] / 7)
    
    # ====================== F. 交易日标识 ======================
    raw_df['is_trading_day'] = np.where(raw_df['day_of_week'].isin([5, 6]), 0, 1)
    
    # ====================== G. 复杂技术指标 ======================
    # MACD指标（12, 26, 9）
    close_price = raw_df['close']
    ema12 = close_price.rolling(window=12).mean()
    ema26 = close_price.rolling(window=26).mean()
    raw_df['macd'] = ema12 - ema26
    raw_df['macd_signal'] = raw_df['macd'].rolling(window=9).mean()
    raw_df['macd_diff'] = raw_df['macd'] - raw_df['macd_signal']
    
    # 动量指标（14期）
    n = 14
    raw_df['momentum'] = raw_df['close'] / raw_df['close'].shift(n) - 1
    
    # ====================== H. 量价交叉特征 ======================
    raw_df['volatility_vol'] = raw_df['price_range'] * raw_df['volume']
    raw_df['return_vol_corr'] = raw_df['price_change_pct'].rolling(window=5).corr(raw_df['volume_change'])
    
    # ====================== I. 数据清洗（与训练时一致） ======================
    # 删除包含NaN的行（由于rolling计算产生）
    processed_df = raw_df.dropna().reset_index()
    
    # ====================== J. 筛选目标特征（按训练时的特征列表） ======================
    # 确保特征顺序和名称与训练时完全一致
    features_df = processed_df[EXPECTED_FEATURES].copy()
    
    return features_df


In [92]:
features = process_raw_data(raw_data)
ml_prediction = best_model.predict(features)

C:\Users\zyc\AppData\Local\Temp\ipykernel_18436\4922374.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_df['datetime'] = pd.to_datetime(raw_df['timestamp'], unit='ms')
C:\Users\zyc\AppData\Local\Temp\ipykernel_18436\4922374.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_df['price_range'] = raw_df['high'] - raw_df['low']
C:\Users\zyc\AppData\Local\Temp\ipykernel_18436\4922374.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[r

TypeError: The DType <class 'numpy.dtype[datetime64]'> could not be promoted by <class 'numpy.dtype[float64]'>. This means that no common DType exists for the given inputs. For example they cannot be stored in a single array unless the dtype is `object`. The full list of DTypes is: (<class 'numpy.dtype[datetime64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>)

In [93]:
features

,datetime,open,high,low,close,volume
0,2025-01-01 05:00:00,93553.91,93937.19,93547.40,93792.02,209.17940
1,2025-01-01 06:00:00,93792.02,93792.02,93568.38,93757.58,272.27846
2,2025-01-01 07:00:00,93757.57,93800.00,93664.08,93684.10,197.93458
3,2025-01-01 08:00:00,93684.11,93756.02,93304.00,93428.46,481.71173
4,2025-01-01 09:00:00,93428.47,93486.29,92888.00,93413.06,798.49773
...,...,...,...,...,...,...
2150,2025-03-31 19:00:00,83291.42,83443.38,82400.00,82462.27,798.20925
2151,2025-03-31 20:00:00,82462.26,82846.20,82350.84,82431.32,701.19812
2152,2025-03-31 21:00:00,82431.32,82752.18,82422.00,82563.99,342.52865
2153,2025-03-31 22:00:00,82563.99,82571.98,82265.10,82409.99,260.87647


In [78]:
y=features['volume']

In [205]:
import numpy as np

def calculate_regression_metrics(prediction, y):
    """
    计算回归模型的5种评估指标：MSE、RMSE、MAE、MAPE、R²
    
    参数:
    prediction (array-like): 模型预测值
    y (array-like): 真实值
    
    返回:
    metrics (dict): 包含各指标的字典
    """
    # 转换为numpy数组并确保维度一致
    prediction = np.array(prediction).flatten()
    y = np.array(y).flatten()
    
    # 检查数据长度是否一致
    if len(prediction) != len(y):
        raise ValueError("预测值和真实值的长度必须一致")
    
    # 1. 均方误差 (MSE)
    mse = np.mean((y - prediction) ** 2)
    
    # 2. 均方根误差 (RMSE)
    rmse = np.sqrt(mse)
    
    # 3. 平均绝对误差 (MAE)
    mae = np.mean(np.abs(y - prediction))
    
    # 4. 平均绝对百分比误差 (MAPE)
    # 处理真实值为0的情况（避免除以0，此处将0值对应的项设为0）
    absolute_percentage_errors = np.where(y == 0, 0, np.abs((y - prediction) / y))
    mape = np.mean(absolute_percentage_errors) * 100  # 转换为百分比
    
    # 5. R平方 (R²)
    y_mean = np.mean(y)
    sst = np.sum((y - y_mean) ** 2)  # 总平方和
    sse = np.sum((y - prediction) ** 2)  # 残差平方和
    r2 = 1 - (sse / sst) if sst != 0 else 0  # 处理sst为0的极端情况（所有真实值相同）
    
    return {
        "MSE": round(mse, 4),
        "RMSE": round(rmse, 4),
        "MAE": round(mae, 4),
        "MAPE": round(mape, 4),
        "R²": round(r2, 4)
    }



In [138]:
# 示例用法
if __name__ == "__main__":
    # 假设预测值和真实值（以比特币交易量为例，单位：万枚）
    y_true = y
    y_pred = ml_prediction
    
    metrics_ML = calculate_regression_metrics(y_pred, y_true)
    print("评估指标结果：")
    for metric, value in metrics_ML.items():
        print(f"{metric}: {value}")

    # 输出示例：
    # 评估指标结果：
    # MSE: 0.1925
    # RMSE: 0.4387
    # MAE: 0.3375
    # MAPE: 2.6923
    # R²: 0.8956


评估指标结果：
MSE: 252477.9503
RMSE: 502.4718
MAE: 274.198
MAPE: 37.8894
R²: 0.8483


# DL

In [50]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

In [46]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout=0.2):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, 
                           batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        batch_size = x.size(0)
        
        # 初始化隐藏状态和细胞状态
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(x.device)
        
        # LSTM forward
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

# 更新模型参数
model_params = {
    'input_size': 13,  # 13个特征
    'hidden_size': 64,
    'num_layers': 2,
    'output_size': 1,  # 预测一个时间点的交易量
    'dropout': 0.2
}


In [191]:
import pandas as pd
import numpy as np
from ta.trend import SMAIndicator, EMAIndicator, MACD
from ta.momentum import RSIIndicator
from ta.volatility import BollingerBands
from sklearn.preprocessing import MinMaxScaler

def add_technical_indicators(df):
    """添加技术指标"""
    # 确保数据按时间排序
    df = df.sort_values('timestamp').reset_index(drop=True)
    
    # 基础特征
    df['price_range'] = df['high'] - df['low']
    df['price_change_pct'] = df['close'].pct_change() * 100
    df['volume_change'] = df['volume'].pct_change()
    
    # 移动平均
    df['sma_4'] = df['close'].rolling(window=4).mean()
    df['volume_sma_4'] = df['volume'].rolling(window=4).mean()
    df['volume_std_4'] = df['volume'].rolling(window=4).std()
    
    # 时间特征
    df['hour'] = pd.to_datetime(df['timestamp'], unit='ms').dt.hour
    df['day_of_week'] = pd.to_datetime(df['timestamp'], unit='ms').dt.dayofweek
    
    # 周期性编码
    df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
    df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
    df['dayofweek_sin'] = np.sin(2 * np.pi * df['day_of_week'] / 7)
    df['dayofweek_cos'] = np.cos(2 * np.pi * df['day_of_week'] / 7)
    
    # MACD
    macd = MACD(close=df['close'])
    df['macd'] = macd.macd()
    df['macd_signal'] = macd.macd_signal()
    df['macd_diff'] = macd.macd_diff()
    
    # RSI
    rsi = RSIIndicator(close=df['close'])
    df['rsi'] = rsi.rsi()
    
    # 布林带
    bb = BollingerBands(close=df['close'])
    df['bb_high'] = bb.bollinger_hband()
    df['bb_low'] = bb.bollinger_lband()
    
    # 删除NaN值
    df = df.dropna()
    
    return df

def load_and_process_data(file_path):
    """加载并处理数据"""
    df = df_1h
    df = add_technical_indicators(df)
    
    feature_columns = [
        'open', 'high', 'low', 'close', 'volume',
        'price_range', 'price_change_pct', 'sma_4',
        'volume_change', 'volume_sma_4', 'volume_std_4',
        'hour_sin', 'hour_cos', 'dayofweek_sin', 'dayofweek_cos',
        'macd', 'macd_signal', 'macd_diff', 'rsi',
        'bb_high', 'bb_low'
    ]
    
    return df[feature_columns]

def series_to_supervised(data, features, n_in=3, n_out=2, dropnan=True):
    """将时间序列转换为监督学习格式"""
    n_vars = len(features)
    df = pd.DataFrame(data)
    cols, names = [], []

    # 输入序列 (t-n, ..., t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [f'{features[j]}(t-{i})' for j in range(n_vars)]

    # 预测序列 (t, t+1, ..., t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [f'{features[j]}(t)' for j in range(n_vars)]
        else:
            names += [f'{features[j]}(t+{i})' for j in range(n_vars)]

    agg = pd.concat(cols, axis=1)
    agg.columns = names
    if dropnan:
        agg.dropna(inplace=True)
    return agg

def process_data(n_in=3, n_out=1):
    """处理数据并生成特征"""
    # 读取数据
    df = raw_data
    
    # 转换时间戳为datetime
    df['datetime'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('datetime', inplace=True)
    
    # 基础特征
    features = ['open', 'high', 'low', 'close', 'volume']
    
    # 添加时间特征
    df['hour'] = df.index.hour
    df['day_of_week'] = df.index.dayofweek  # 添加星期几特征
    df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)  # 添加是否周末特征
    
    # 交易量特征
    df['volume_sma_4'] = df['volume'].rolling(window=4).mean()  # 4小时移动平均
    df['volume_std_4'] = df['volume'].rolling(window=4).std()   # 4小时标准差
    df['volume_momentum'] = df['volume'] / df['volume'].rolling(24).mean()  # 相对于24小时平均的动量
    
    # 价格和波动性特征
    df['price_change'] = df['close'].pct_change()
    df['volatility'] = df['price_change'].rolling(24).std()  # 24小时波动率
    
    # 更新特征列表
    features = features + [
        'hour', 'day_of_week', 'is_weekend',
        'volume_sma_4', 'volume_std_4', 'volume_momentum',
        'price_change', 'volatility'
    ]
    
    # 处理缺失值
    df = df.ffill()
    df = df.dropna()
    
    # 标准化数据
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(df[features])
    
    # 创建序列数据
    X, y = [], []
    time_nodes=[]
    for i in range(len(scaled_data) - n_in - n_out + 1):
        X.append(scaled_data[i:(i + n_in)])
        y.append(scaled_data[i + n_in, 4])  # 预测volume（第5列）
        time_nodes.append(df.index[i + n_in])
    
    X = np.array(X)
    y = np.array(y)
    
    print(f"X shape: {X.shape}, y shape: {y.shape}")
    print(f"特征列表: {features}")
    
    return X, y, scaler,time_nodes

if __name__ == "__main__":
    # 测试数据处理

    X_hourly, y_hourly, scaler_hourly,timenodes = process_data()
    print("\n处理结果:")
    print(f"X shape: {X_hourly.shape}")
    print(f"y shape: {y_hourly.shape}") 

C:\Users\zyc\AppData\Local\Temp\ipykernel_18436\3540350812.py:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['datetime'] = pd.to_datetime(df['timestamp'], unit='ms')
C:\Users\zyc\AppData\Local\Temp\ipykernel_18436\3540350812.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['hour'] = df.index.hour
C:\Users\zyc\AppData\Local\Temp\ipykernel_18436\3540350812.py:108: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

X shape: (2152, 3, 13), y shape: (2152,)
特征列表: ['open', 'high', 'low', 'close', 'volume', 'hour', 'day_of_week', 'is_weekend', 'volume_sma_4', 'volume_std_4', 'volume_momentum', 'price_change', 'volatility']

处理结果:
X shape: (2152, 3, 13)
y shape: (2152,)


In [192]:
def predict_next_hour_volume(model, data, scaler):
    """预测下一个小时的交易量"""
    model.eval()
    with torch.no_grad():
        # 转换输入数据为张量
        X = torch.FloatTensor(data).unsqueeze(0)  # 添加batch维度
        if torch.cuda.is_available():
            X = X.cuda()
            model = model.cuda()
        
        # 预测
        pred = model(X)
        pred = pred.cpu().numpy()
        
        # 反标准化得到实际交易量
        pred_full = np.zeros((len(pred), 13))
        pred_full[:, 4] = pred.reshape(-1)  # volume是第5个特征
        pred_volume = scaler.inverse_transform(pred_full)[:, 4]
        
        return pred_volume[0]

In [195]:
    # 1. 加载模型
    model = LSTMModel(
        input_size=13,  # 特征数量
        hidden_size=64,
        num_layers=2,
        output_size=1,
        dropout=0.2
    )
    model.load_state_dict(torch.load('D:/Study_Master/TERM2/FTEC5530/ftec5530-main 1/dapplearning/model/best_model.pth'))
    model.eval()
    
    # 2. 加载测试数据
    test_data_path = df_1h
    X_test, y_test, scaler,timenodes = process_data(n_in=3, n_out=1)
    

    
    # 4. 进行预测
    predictions = []
    actuals = []
    
    print("\n开始预测...")
    for i in range(len(X_test)):
        # 预测下一个小时的交易量
        pred_volume = predict_next_hour_volume(model, X_test[i], scaler)
        
        # 修改这部分代码
        actual_full = np.zeros((1, 13))
        actual_full[0, 4] = y_test[i]  # volume是第5个特征
        actual_volume = scaler.inverse_transform(actual_full)[0, 4]
        
        predictions.append(pred_volume)
        actuals.append(actual_volume)
        
        if i % 100 == 0:
            print(f"已完成 {i}/{len(X_test)} 个预测")
    

C:\Users\zyc\AppData\Local\Temp\ipykernel_18436\3540350812.py:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['datetime'] = pd.to_datetime(df['timestamp'], unit='ms')
C:\Users\zyc\AppData\Local\Temp\ipykernel_18436\3540350812.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['hour'] = df.index.hour
C:\Users\zyc\AppData\Local\Temp\ipykernel_18436\3540350812.py:108: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

X shape: (2152, 3, 13), y shape: (2152,)
特征列表: ['open', 'high', 'low', 'close', 'volume', 'hour', 'day_of_week', 'is_weekend', 'volume_sma_4', 'volume_std_4', 'volume_momentum', 'price_change', 'volatility']

开始预测...
已完成 0/2152 个预测
已完成 100/2152 个预测
已完成 200/2152 个预测
已完成 300/2152 个预测
已完成 400/2152 个预测
已完成 500/2152 个预测
已完成 600/2152 个预测
已完成 700/2152 个预测
已完成 800/2152 个预测
已完成 900/2152 个预测
已完成 1000/2152 个预测
已完成 1100/2152 个预测
已完成 1200/2152 个预测
已完成 1300/2152 个预测
已完成 1400/2152 个预测
已完成 1500/2152 个预测
已完成 1600/2152 个预测
已完成 1700/2152 个预测
已完成 1800/2152 个预测
已完成 1900/2152 个预测
已完成 2000/2152 个预测
已完成 2100/2152 个预测


In [206]:
# 假设预测值和真实值（以比特币交易量为例，单位：万枚）
y_true = actuals
y_pred = predictions

metrics_DL = calculate_regression_metrics(y_pred, y_true)
print("评估指标结果：")
for metric, value in metrics_DL.items():
    print(f"{metric}: {value}")

评估指标结果：
MSE: 860708.3827
RMSE: 927.7437
MAE: 436.2098
MAPE: 36.3662
R²: 0.4833


In [213]:
import numpy as np
y_true = np.array(actuals)
y_pred = np.array(predictions)

# 计算对数误差
log_errors = np.abs(np.log(y_pred / y_true))

# 计算平均、最大、最小对数误差
mean_error = np.mean(log_errors)
max_error = log_errors.max()
min_error = log_errors.min()

# 打印结果
print(f"平均对数误差: {mean_error:.4f}")
print(f"最大对数误差: {max_error:.4f}")
print(f"最小对数误差: {min_error:.4f}")

平均对数误差: 0.3544
最大对数误差: 2.5751
最小对数误差: 0.0003


# combined method

In [113]:
metrics_ML

{'MSE': 252477.9503,
 'RMSE': 502.4718,
 'MAE': 274.198,
 'MAPE': 37.8894,
 'R²': 0.8483}

In [207]:
metrics_DL

{'MSE': 860708.3827,
 'RMSE': 927.7437,
 'MAE': 436.2098,
 'MAPE': 36.3662,
 'R²': 0.4833}

In [156]:
features['predictions'] = ml_prediction

# 2. 重命名 `volume` 列为 `actuals`
features = features.rename(columns={'volume': 'actuals'})

# 3. 调整列顺序为 `datetime`, `open`, `high`, `low`, `close`, `predictions`, `actuals`
new_column_order = ['datetime', 'open', 'high', 'low', 'close', 'predictions', 'actuals']
ML = features[['datetime', 'predictions', 'actuals']]

In [157]:
ML

,datetime,predictions,actuals
0,2025-01-01 05:00:00,436.257203,209.17940
1,2025-01-01 06:00:00,458.575898,272.27846
2,2025-01-01 07:00:00,402.512867,197.93458
3,2025-01-01 08:00:00,737.518758,481.71173
4,2025-01-01 09:00:00,874.849017,798.49773
...,...,...,...
2150,2025-03-31 19:00:00,1340.553883,798.20925
2151,2025-03-31 20:00:00,899.563694,701.19812
2152,2025-03-31 21:00:00,566.069659,342.52865
2153,2025-03-31 22:00:00,532.326591,260.87647


In [159]:
# 1. 处理 datetime：每个样本对应一个时间点（假设为最后一个时间步的时间）
datetime_series = pd.Series(timenodes, name='datetime')

# 3. 处理预测值和实际值（假设长度均为样本数）
predictions_series = pd.Series(predictions, name='predictions')
actuals_series = pd.Series(actuals, name='actuals')  # 若实际值存在，需与样本数对齐

DL = pd.concat(
    [datetime_series,  predictions_series, actuals_series], 
    axis=1
)


In [160]:
DL

,datetime,predictions,actuals
0,2025-01-01 08:00:00,419.504336,481.71173
1,2025-01-01 09:00:00,495.957025,798.49773
2,2025-01-01 10:00:00,680.034703,265.55192
3,2025-01-01 11:00:00,557.697234,321.72225
4,2025-01-01 12:00:00,567.128701,499.35613
...,...,...,...
2147,2025-03-31 19:00:00,853.978448,798.20925
2148,2025-03-31 20:00:00,893.243333,701.19812
2149,2025-03-31 21:00:00,691.303453,342.52865
2150,2025-03-31 22:00:00,557.659453,260.87647


In [218]:
w1 = 0.8
w2 = 1 - w1

# 合并数据框，处理缺失值
merged = pd.merge(ML, DL, on='datetime', suffixes=('_ML', '_DL'), how='outer')
merged['predictions_ML'] = merged['predictions_ML'].fillna(0)
merged['predictions_DL'] = merged['predictions_DL'].fillna(0)

# 计算加权混合后的预测值
merged['combined_prediction'] = merged['predictions_ML'] * w1 + merged['predictions_DL'] * w2
# 4. 关键逻辑：条件选择actuals（优先使用ML的actuals，若缺失则用DL的）
    # 使用pandas.where实现：若actuals_ML不为NaN，取该值；否则取actuals_DL的值
merged['combined_actuals'] = merged['actuals_ML'].where(
    merged['actuals_ML'].notna(),  # 条件：ML的actuals存在（非NaN）
    other=merged['actuals_DL']     # 否则使用DL的actuals
)

# 选取结果列（可根据需求调整）
result = merged[['datetime', 'combined_prediction','combined_actuals', 'predictions_ML', 'predictions_DL']]

print(result)

                datetime  combined_prediction  combined_actuals  \
0    2025-01-01 05:00:00           349.005762         209.17940   
1    2025-01-01 06:00:00           366.860718         272.27846   
2    2025-01-01 07:00:00           322.010293         197.93458   
3    2025-01-01 08:00:00           673.915874         481.71173   
4    2025-01-01 09:00:00           799.070618         798.49773   
...                  ...                  ...               ...   
2150 2025-03-31 19:00:00          1243.238796         798.20925   
2151 2025-03-31 20:00:00           898.299622         701.19812   
2152 2025-03-31 21:00:00           591.116418         342.52865   
2153 2025-03-31 22:00:00           537.393163         260.87647   
2154 2025-03-31 23:00:00           646.425502         383.74846   

      predictions_ML  predictions_DL  
0         436.257203        0.000000  
1         458.575898        0.000000  
2         402.512867        0.000000  
3         737.518758      419.504336  


In [219]:
# 假设预测值和真实值（以比特币交易量为例，单位：万枚）
y_true = result['combined_actuals'] 
y_pred = result['combined_prediction']

metrics_DL = calculate_regression_metrics(y_pred, y_true)
print("评估指标结果：")
for metric, value in metrics_DL.items():
    print(f"{metric}: {value}")

评估指标结果：
MSE: 284951.17
RMSE: 533.8082
MAE: 265.6674
MAPE: 34.519
R²: 0.8288


In [230]:
import numpy as np
import pandas as pd

merged = pd.merge(ML, DL, on='datetime', suffixes=('_ML', '_DL'), how='outer')

# 确保数据按时间排序
merged = merged.sort_values('datetime')
merged['combined_actuals'] = merged['actuals_ML'].where(
    merged['actuals_ML'].notna(),  # 条件：ML的actuals存在（非NaN）
    other=merged['actuals_DL']     # 否则使用DL的actuals
)

best_w1 = 0
min_score = float('inf')

for w1 in np.arange(0, 1.01, 0.01):
    w2 = 1 - w1
    combined_prediction = merged['predictions_ML'] * w1 + merged['predictions_DL'] * w2
    
    # 计算 RMSE
    rmse = np.sqrt(np.mean((combined_prediction - merged['combined_actuals'])**2))
    
    # 计算 SMAPE（处理分母为 0 的情况）
    numerator = np.abs(merged['combined_actuals'] - combined_prediction)
    denominator = (np.abs(merged['combined_actuals']) + np.abs(combined_prediction)) / 2
    denominator = np.where(denominator == 0, 1, denominator)  # 避免除以 0
    smape = np.mean(numerator / denominator) * 100
    
    # 计算方向准确率（处理 diff 后的 NaN）
    actual_diff = merged['combined_actuals'].diff() > 0
    pred_diff = combined_prediction.diff() > 0
    actual_diff = actual_diff.dropna()
    pred_diff = pred_diff.dropna()
    direction_accuracy = (actual_diff == pred_diff).mean()
    
    # 计算 R²
    ss_res = np.sum((merged['combined_actuals'] - combined_prediction)**2)
    ss_tot = np.sum((merged['combined_actuals'] - merged['combined_actuals'].mean())**2)
    r2 = 1 - (ss_res / ss_tot)
    
    # 计算综合指标
    score = 0.35 * rmse + 0.3 * smape + 0.25 * (1 - direction_accuracy) + 0.1 * (1 - r2)
    print(rmse)
    print(smape)
    print(direction_accuracy)
    print(r2)
    
    if score < min_score:
        min_score = score
        best_w1 = w1

print(f"最佳权重 w1: {best_w1}")

927.7437052970745
34.123288312241954
0.46960556844547563
0.4837095111214238
921.4772234226484
33.86593729402213
0.48306264501160096
0.4906605653969025
915.2286333577878
33.614155406511514
0.4877030162412993
0.49754486459043956
908.9983040754776
33.36606890523112
0.491415313225058
0.504362408702035
902.7866136408437
33.121080637619656
0.4988399071925754
0.5111131977316885
896.5939494492438
32.882311786915665
0.5020881670533642
0.5177972316794004
890.4207084697456
32.648270401480474
0.5109048723897912
0.5244145105451705
884.2672974940383
32.418550494579556
0.5178654292343388
0.5309650343289989
878.1341333907609
32.192676790397165
0.5243619489559165
0.5374488030308855
872.0216433652025
31.97079895996446
0.5317865429234339
0.5438658166508306
865.9302652244215
31.75255153081376
0.5392111368909512
0.5502160751888336
859.8604476476436
31.53875771947703
0.5443155452436195
0.5564995786448952
853.812650461948
31.328808952623227
0.5517401392111368
0.5627163270190148
847.7873449231133
31.122021892

In [259]:
merged = pd.merge(ML, DL, on='datetime', suffixes=('_ML', '_DL'), how='outer')

# 确保数据按时间排序
merged = merged.sort_values('datetime')
merged['combined_actuals'] = merged['actuals_ML'].where(
    merged['actuals_ML'].notna(),  # 条件：ML的actuals存在（非NaN）
    other=merged['actuals_DL']     # 否则使用DL的actuals
)

# 确保数据按时间排序
merged = merged.sort_values('datetime')

rmse_list = []
smape_list = []
direction_accuracy_list = []
r2_list = []

# 收集所有权重下的指标值
for w1 in np.arange(0, 1.01, 0.01):
    w2 = 1 - w1
    combined_prediction = merged['predictions_ML'] * w1 + merged['predictions_DL'] * w2
    
    # 计算 RMSE
    rmse = np.sqrt(np.mean((combined_prediction - merged['combined_actuals'])**2))
    rmse_list.append(rmse)
    
    # 计算 SMAPE（处理分母为 0 的情况）
    numerator = np.abs(merged['combined_actuals'] - combined_prediction)
    denominator = (np.abs(merged['combined_actuals']) + np.abs(combined_prediction)) / 2
    denominator = np.where(denominator == 0, 1, denominator)  # 避免除以 0
    smape = np.mean(numerator / denominator) * 100
    smape_list.append(smape)
    
    # 计算方向准确率（处理 diff 后的 NaN）
    actual_diff = merged['combined_actuals'].diff() > 0
    pred_diff = combined_prediction.diff() > 0
    actual_diff = actual_diff.dropna()
    pred_diff = pred_diff.dropna()
    direction_accuracy = (actual_diff == pred_diff).mean()
    direction_accuracy_list.append(direction_accuracy)
    
    # 计算 R²
    ss_res = np.sum((merged['combined_actuals'] - combined_prediction)**2)
    ss_tot = np.sum((merged['combined_actuals'] - merged['combined_actuals'].mean())**2)
    r2 = 1 - (ss_res / ss_tot)
    r2_list.append(r2)

# 计算各指标的最值
rmse_min, rmse_max = np.min(rmse_list), np.max(rmse_list)
smape_min, smape_max = np.min(smape_list), np.max(smape_list)
dir_min, dir_max = np.min(direction_accuracy_list), np.max(direction_accuracy_list)
r2_min, r2_max = np.min(r2_list), np.max(r2_list)

best_w1 = 0
max_score = 0

# 重新遍历计算归一化后的综合指标
for i, w1 in enumerate(np.arange(0, 1.01, 0.01)):
    # 归一化处理
    
    rmse_normalized = 1 - (rmse_list[i] - rmse_min) / (rmse_max - rmse_min)  # RMSE越小越好，转换为越大越优
    smape_normalized = 1 - (smape_list[i] - smape_min) / (smape_max - smape_min)  # SMAPE越小越好，转换为越大越优
    dir_normalized = (direction_accuracy_list[i] - dir_min) / (dir_max - dir_min)  # 方向准确率越大越好
    r2_normalized = (r2_list[i] - r2_min) / (r2_max - r2_min)  # R²越大越好
    
    # 按权重计算综合指标（场景1权重）
    score = 0.4 * rmse_normalized + 0.3 * smape_normalized + 0.2 * dir_normalized + 0.1 * r2_normalized
    if w1==0 or w1==1:
        print(f"w1: {w1}")
        print(f"rmse_normalized: {rmse_list[i]}   {rmse_normalized}")
        print(f"smape_normalized: {smape_normalized}")
        print(f"dir_normalized: {dir_normalized}")
        print(f"r2_normalized: {r2_normalized}")
        print(f"score: {score}")

    
    if score > max_score:  # 因为指标都归一化为越大越优，找最大值
        max_score = score
        best_w1 = w1

print(f"最佳权重 w1: {best_w1}")
print(f"最佳score w1: {max_score}")

w1: 0.0
rmse_normalized: 927.7437052970745   0.0
smape_normalized: 0.0
dir_normalized: 0.0
r2_normalized: 0.0
score: 0.0
w1: 1.0
rmse_normalized: 502.7627480339796   1.0
smape_normalized: 0.622996047584003
dir_normalized: 0.9978260869565218
r2_normalized: 1.0
score: 0.8864640316665053
最佳权重 w1: 0.8
最佳score w1: 0.9244574937251733
